In [18]:
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
import re

# Initialiser le driver
driver = webdriver.Chrome()

# Listes pour stocker les données
books_total = []
auteurs_total = []
list_total = []

# URLs des différentes listes
urls = [
    'https://www.senscritique.com/liste/sophia_perennis/3440052',
    'https://www.senscritique.com/liste/developpement_personnel/2336516?page=1',
    #'https://www.senscritique.com/liste/voir/1818898?page=1',
    #'https://www.senscritique.com/liste/livres_que_j_ai_aimes_osho_rajneesh/961827?page=1',
    #'https://www.senscritique.com/liste/un_chemin_spirituel/1695163?page=1',
    #'https://www.senscritique.com/liste/la_bibliotheque_ideale_4_selon_pacome_thiellement/1567315?page=1',
    #'https://www.senscritique.com/liste/le_traditionalisme/2979680?page=1',
    #'https://www.senscritique.com/liste/la_gnose/3143468?page=1',
    #'https://www.senscritique.com/liste/resonnances_mystiques/3143118?page=1',
    #'https://www.senscritique.com/liste/097_tradition_primordiale/3313896?page=1',
    #'https://www.senscritique.com/liste/etats_de_conscience/2027111?page=1',
]


# Gérer les formats différents, avec ou sans query parameter
for i in range(len(urls)):
    if not re.search(r'page', urls[i][-6:]):  # Vérifie si "page" est absent des 6 derniers caractères
        urls[i] += '?page=1'
        
# Préparer les URLs pour la pagination    
for url in urls:
    urls_1 = [url[:-1] for url in urls]

try:
    consent_clicked = False  # Flag pour indiquer si le bouton consentement a été cliqué

    for url in urls_1:
        # Extraire le nom de la liste depuis l'URL
        list_name_match = re.search(r"/liste/([^/]+)/", url)
        if list_name_match:
            list_name = list_name_match.group(1)
        else:
            list_name = "Unknown"

        for i in range(1, 101):  # On suppose un maximum de 100 pages
            # Charger la page
            driver.get(f"{url}{i}")

            # Gérer la pop-up de consentement uniquement sur la première page
            if not consent_clicked:
                try:
                    consent_button = WebDriverWait(driver, 10).until(
                        EC.element_to_be_clickable((By.ID, "didomi-notice-agree-button"))
                    )
                    consent_button.click()
                    consent_clicked = True  # Marquer le bouton comme cliqué
                    print("Consent button clicked.")
                except Exception as e:
                    print("Consent button not found or already clicked.")

            # Vérifier s'il y a du contenu sur la page (par exemple, vérifier s'il y a des livres)
            try:
                book_author_containers = WebDriverWait(driver, 3).until(
                    EC.presence_of_all_elements_located(
                        (By.CLASS_NAME, 'sc-e6f263fc-0.sc-47f61275-0.fzmpxB.fyVVLV.sc-6b8f8da3-6.jLINWO')
                    )
                )
            except Exception as e:
                print(f"No content found on page {i}. Ending loop.")
                break

            # Extraire les titres et auteurs
            for container in book_author_containers:
                # Extraire les auteurs pour chaque livre
                author_links = container.find_elements(By.CSS_SELECTOR, 'a[data-testid="link"]')
                book_authors = [link.text for link in author_links]

                # Ajouter à la liste des auteurs pour cette page
                auteurs_total.append(book_authors[0] if len(book_authors) == 1 else book_authors)
                list_total.append(list_name)  # Ajouter le nom de la liste

            # Extraire les titres des livres
            h3_elements = driver.find_elements(By.TAG_NAME, "h3")
            titles_list = [h3.text for h3 in h3_elements]
            books_total.extend(titles_list)
            

            print(f"Page {i} processed for list '{list_name}'. Found {len(titles_list)} books.")

finally:
    # Fermer le driver à la fin
    driver.quit()

# Afficher les résultats finaux
print("Books:", books_total)
print("Authors:", auteurs_total)
print("Liste:", list_total)


Consent button clicked.
No content found on page 1. Ending loop.
No content found on page 1. Ending loop.
Books: []
Authors: []
Liste: []


In [15]:
import re

urls = [
    'https://www.senscritique.com/liste/sophia_perennis/3440052',
    'https://www.senscritique.com/liste/developpement_personnel/2336516?page=1'
]

print("Avant modification :", urls)

# Mettre à jour la liste en modifiant les éléments directement
for i in range(len(urls)):
    if not re.search(r'page', urls[i][-6:]):  # Vérifie si "page" est absent des 6 derniers caractères
        urls[i] += '?page=1'

print("Après modification :", urls)


Avant modification : ['https://www.senscritique.com/liste/sophia_perennis/3440052', 'https://www.senscritique.com/liste/developpement_personnel/2336516?page=1']
Après modification : ['https://www.senscritique.com/liste/sophia_perennis/3440052?page=1', 'https://www.senscritique.com/liste/developpement_personnel/2336516?page=1']


In [46]:

df = pd.DataFrame({'Titres':books_total,
                   'Auteur(s)':auteurs_total,
                  'Liste':list_total})

display(df)

df.to_csv("liste_pour_riheme.csv")

,Titres,Auteur(s),Liste
0,Homo Mythologicus (2022),Pierre-Yves Lenoble,sophia_perennis
1,Orient et Occident (1924),René Guénon,sophia_perennis
2,Le symbolisme de la croix (1931),René Guénon,sophia_perennis
3,Autorité spirituelle et pouvoir temporel (1929),René Guénon,sophia_perennis
4,De l'unité transcendante des religions (1948),Frithjof Schuon,sophia_perennis
...,...,...,...
320,Saturne (2020),Sarah Chiche,la_bibliotheque_ideale_4_selon_pacome_thiellement
321,Kétamine (2020),Zoé Sagan,la_bibliotheque_ideale_4_selon_pacome_thiellement
322,Le Cœur synthétique (2020),Chloé Delaume,la_bibliotheque_ideale_4_selon_pacome_thiellement
323,L'Evangile selon Nick Cave : Le Gospel de l'âg...,Arthur-Louis Cingualte,la_bibliotheque_ideale_4_selon_pacome_thiellement


In [62]:
# Copier le DataFrame
df2 = df.copy()

# Convertir les listes d'auteurs en une chaîne séparée par des virgules
df2["Auteur(s)"] = df2["Auteur(s)"].apply(lambda x: ", ".join(x) if isinstance(x, list) else x)


# Compter les occurrences de chaque combinaison "Titres" et "Auteur(s)"
df2["recensé"] = df2.groupby(['Titres', 'Auteur(s)'])['Titres'].transform('count')

# Grouper par "Titres" et "Auteur(s)" pour combiner les listes et supprimer les doublons
df_unique = df2.groupby(['Titres', 'Auteur(s)']).agg({
    'Liste': lambda x: ", ".join(sorted(set(x))),  # Combine les listes uniques, triées
    'recensé': 'first'  # Conserve le nombre recensé
}).reset_index()

# Afficher le DataFrame final
display(df_unique.sort_values('recensé', ascending=False))


,Titres,Auteur(s),Liste,recensé
305,Voir (1971),Carlos Castaneda,la_bibliotheque_ideale_4_selon_pacome_thiellem...,3
122,La Bible,,"livres_que_j_ai_aimes_osho_rajneesh, sophia_pe...",2
37,De l'autre côté du miroir (1871),Lewis Carroll,la_bibliotheque_ideale_4_selon_pacome_thiellem...,2
208,Les Aventures d'Alice au pays des merveilles (...,Lewis Carroll,la_bibliotheque_ideale_4_selon_pacome_thiellem...,2
135,La République,Platon,sophia_perennis,2
...,...,...,...,...
103,L'exégèse,Philip K. Dick,la_bibliotheque_ideale_4_selon_pacome_thiellement,1
102,L'extase et l'agonie (1983),Irving Stone,livres_que_j_ai_aimes_osho_rajneesh,1
101,L'errant,Khalil Gibran,livres_que_j_ai_aimes_osho_rajneesh,1
100,L'art de la paix (1992),Morihei Ueshiba,voir,1


In [68]:
display(df2.groupby(['Titres', 'Auteur(s)'].agg({'Liste':lambda x: x})))

AttributeError: 'list' object has no attribute 'agg'

In [83]:
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
import re
import numpy as np

# Initialiser le driver
driver = webdriver.Chrome()

# Listes pour stocker les données
books_total = []
auteurs_total = []
list_total = []
notes_total = []

# URLs des différentes listes
urls = [
    'https://www.senscritique.com/liste/sophia_perennis/3440052?page=1',
    'https://www.senscritique.com/liste/developpement_personnel/2336516?page=1',
    'https://www.senscritique.com/liste/voir/1818898?page=1',
    'https://www.senscritique.com/liste/livres_que_j_ai_aimes_osho_rajneesh/961827?page=1',
    'https://www.senscritique.com/liste/un_chemin_spirituel/1695163?page=1',
    'https://www.senscritique.com/liste/la_bibliotheque_ideale_4_selon_pacome_thiellement/1567315?page=1',
    'https://www.senscritique.com/liste/le_traditionalisme/2979680?page=1',
    'https://www.senscritique.com/liste/la_gnose/3143468?page=1',
    'https://www.senscritique.com/liste/resonnances_mystiques/3143118?page=1',
    'https://www.senscritique.com/liste/097_tradition_primordiale/3313896?page=1',
    'https://www.senscritique.com/liste/etats_de_conscience/2027111?page=1',
]

# Préparer les URLs pour la pagination
urls_1 = [url[:-1] for url in urls]

try:
    consent_clicked = False  # Flag pour indiquer si le bouton consentement a été cliqué

    for url in urls_1:
        # Extraire le nom de la liste depuis l'URL
        list_name_match = re.search(r"/liste/([^/]+)/", url)
        if list_name_match:
            list_name = list_name_match.group(1)
        else:
            list_name = "Unknown"

        for i in range(1, 101):  # On suppose un maximum de 100 pages
            # Charger la page
            driver.get(f"{url}{i}")

            # Gérer la pop-up de consentement uniquement sur la première page
            if not consent_clicked:
                try:
                    consent_button = WebDriverWait(driver, 10).until(
                        EC.element_to_be_clickable((By.ID, "didomi-notice-agree-button"))
                    )
                    consent_button.click()
                    consent_clicked = True  # Marquer le bouton comme cliqué
                    print("Consent button clicked.")
                except Exception as e:
                    print("Consent button not found or already clicked.")

            # Vérifier s'il y a du contenu sur la page (par exemple, vérifier s'il y a des livres)
            try:
                book_author_containers = WebDriverWait(driver, 3).until(
                    EC.presence_of_all_elements_located(
                        (By.CLASS_NAME, 'sc-e6f263fc-0.sc-47f61275-0.fzmpxB.fyVVLV.sc-6b8f8da3-6.jLINWO')
                    )
                )
            except Exception as e:
                print(f"No content found on page {i}. Ending loop.")
                break

            # Extraire les titres et auteurs
            for container in book_author_containers:
                # Extraire les auteurs pour chaque livre
                author_links = container.find_elements(By.CSS_SELECTOR, 'a[data-testid="link"]')
                book_authors = [link.text for link in author_links]

                # Ajouter à la liste des auteurs pour cette page
                auteurs_total.append(book_authors[0] if len(book_authors) == 1 else book_authors)
                list_total.append(list_name)  # Ajouter le nom de la liste

            # Extraire les titres des livres
            h3_elements = driver.find_elements(By.TAG_NAME, "h3")
            titles_list = [h3.text for h3 in h3_elements]
            books_total.extend(titles_list)
            
            # Extractions des notes
            books = driver.find_elements(By.CSS_SELECTOR, "div[data-testid='poster']")
            
            for book in books:
                try:
                    rating = book.find_element(By.CSS_SELECTOR, "div[data-testid='Rating']").text
                    notes_total.append(rating)
                except:
                    notes_total.append(np.nan)

            

            print(f"Page {i} processed for list '{list_name}'. Found {len(titles_list)} books.")

finally:
    # Fermer le driver à la fin
    driver.quit()




Consent button clicked.
Page 1 processed for list 'sophia_perennis'. Found 30 books.
Page 2 processed for list 'sophia_perennis'. Found 30 books.
Page 3 processed for list 'sophia_perennis'. Found 25 books.
No content found on page 4. Ending loop.
Page 1 processed for list 'developpement_personnel'. Found 30 books.
Page 2 processed for list 'developpement_personnel'. Found 30 books.
Page 3 processed for list 'developpement_personnel'. Found 30 books.
Page 4 processed for list 'developpement_personnel'. Found 30 books.
Page 5 processed for list 'developpement_personnel'. Found 30 books.
Page 6 processed for list 'developpement_personnel'. Found 30 books.
Page 7 processed for list 'developpement_personnel'. Found 30 books.
Page 8 processed for list 'developpement_personnel'. Found 28 books.
No content found on page 9. Ending loop.
Page 1 processed for list 'voir'. Found 30 books.
Page 2 processed for list 'voir'. Found 25 books.
No content found on page 3. Ending loop.
Page 1 processed f

In [84]:
df = pd.DataFrame({'Titres':books_total,
                   'Auteur(s)':auteurs_total,
                  'Liste':list_total,
                  'Notes':notes_total})

display(df)



,Titres,Auteur(s),Liste,Notes
0,Homo Mythologicus (2022),Pierre-Yves Lenoble,sophia_perennis,NaN
1,Orient et Occident (1924),René Guénon,sophia_perennis,7.3
2,Le symbolisme de la croix (1931),René Guénon,sophia_perennis,7.7
3,Autorité spirituelle et pouvoir temporel (1929),René Guénon,sophia_perennis,8.8
4,De l'unité transcendante des religions (1948),Frithjof Schuon,sophia_perennis,8
...,...,...,...,...
1859,Chaman,Galsan Tschinag,etats_de_conscience,NaN
1860,Traité d'Herboristerie énergétique (2022),"[Matthew Wood, Laure Rose]",etats_de_conscience,NaN
1861,Mère,Laurent Huguelit,etats_de_conscience,7.9
1862,Le chamane et le psy,"[Laurent Huguelit, Olivier Chambon]",etats_de_conscience,NaN


In [87]:
df2 = df.copy()

# Convertir les listes d'auteurs en une chaîne séparée par des virgules
df2["Auteur(s)"] = df2["Auteur(s)"].apply(lambda x: ", ".join(x) if isinstance(x, list) else x)


# Compter les occurrences de chaque combinaison "Titres" et "Auteur(s)"
df2["recensé"] = df2.groupby(['Titres', 'Auteur(s)'])['Titres'].transform('count')

# Grouper par "Titres" et "Auteur(s)" pour combiner les listes et supprimer les doublons
df_unique = df2.groupby(['Titres', 'Auteur(s)']).agg({
    'Liste': lambda x: ", ".join(sorted(set(x))),  # Combine les listes uniques, triées
    'recensé': 'first', # Conserve le nombre recensé
    'Notes': 'first'
}).reset_index()

# Afficher le DataFrame final

df_final = df_unique.sort_values(['recensé', 'Notes'], ascending=False)
display(df_final)



,Titres,Auteur(s),Liste,recensé,Notes
871,Le Règne de la quantité et les signes des temp...,René Guénon,"097_tradition_primordiale, developpement_perso...",7,8.3
98,Chevaucher le tigre (1961),Julius Evola,"097_tradition_primordiale, developpement_perso...",6,7.6
1068,Les États multiples de l'être (1932),René Guénon,"097_tradition_primordiale, developpement_perso...",5,8.3
360,Introduction Générale à l'étude des doctrines ...,René Guénon,"097_tradition_primordiale, developpement_perso...",5,8.2
166,De l'unité transcendante des religions (1948),Frithjof Schuon,"097_tradition_primordiale, developpement_perso...",5,8
...,...,...,...,...,...
1462,Écrits spirituels,Abd El-Kader,resonnances_mystiques,1,None
1476,Ésotérisme guénonien et mystère chrétien (1997),Jean Borella,la_gnose,1,None
1477,Éthique du mikado (2015),Sarah Chiche,la_bibliotheque_ideale_4_selon_pacome_thiellement,1,None
1480,Évangile de la vérité,,la_gnose,1,None


In [88]:
df_final.to_csv("df_final_riheme.csv")